<header style="padding:10px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<p style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>VantageCloud Lake Systems Scaling and Monitoring</b></p>
<hr>

<br>

<b style = 'font-size:24px;font-family:Arial;color:#E37C4D'>Demo 0 - Foreign Table Setup</b>

<p style = 'font-size:16px;font-family:Arial'>This notebook sets up the Foreign Table "retail_sample_data.sales_transaction_line_parquet_ft" that is used for the concurrency scaling query in "Demo 1".  The table uses the existing retail_sample_data.DEMO_AUTH_NOS authorization object and the td-usecases-data-store S3 bucket.</p>
<hr>

In [ ]:
import teradatasql, json

# load vars json
with open('../vars.json', 'r') as f:
    session_vars = json.load(f)

name = session_vars['hierarchy']['SYSDBA']['username']
pwd = session_vars['hierarchy']['SYSDBA']['password']

with teradatasql.connect(host = session_vars['environment']['host'], 
                     user = name, 
                     password = pwd) as con:
    cur = con.cursor()
    try:
        cur.execute('DROP TABLE retail_sample_data.sales_transaction_line_parquet_ft;')
    except Exception as e:
        # Table already exists
        if str(e.args).find("3807") >= 1:
            pass
        else:
            raise
            
    qry = '''
    CREATE MULTISET FOREIGN TABLE retail_sample_data.sales_transaction_line_parquet_ft ,FALLBACK ,
         EXTERNAL SECURITY retail_sample_data.DEMO_AUTH_NOS
         (
          Location VARCHAR(2048) CHARACTER SET UNICODE CASESPECIFIC,
          SalesTranId INTEGER,
          SalesTranLineNum SMALLINT,
          ItemId BIGINT,
          ItemQty SMALLINT,
          UnitSellingPriceAmt DECIMAL(7,4),
          UnitCostAmt DECIMAL(7,4),
          TranLineStatusCd CHAR(1) CHARACTER SET UNICODE NOT CASESPECIFIC,
          SalesTranLineStartDttm TIMESTAMP(6) FORMAT 'Y4-MM-DDBHH:MI:SSDS(6)',
          TranLineSalesTypeCd CHAR(12) CHARACTER SET UNICODE NOT CASESPECIFIC,
          SalesTranLineEndDttm TIMESTAMP(6) FORMAT 'Y4-MM-DDBHH:MI:SSDS(6)',
          TranLineDate DATE FORMAT 'Y4/MM/DD',
          locationid SMALLINT,
          dssupdatetime BYTEINT)
    USING
    (
          LOCATION  ('/s3/s3.amazonaws.com/td-usecases-data-store/retail_sample_data/parquet/sales_transaction_line_gz/')
          MANIFEST  ('FALSE')
          PATHPATTERN  ('$alpha_admin_data_copy/$varchar2/$retail_sample_data/$filename')
          STOREDAS  ('PARQUET')
    )
    NO PRIMARY INDEX 
    PARTITION BY COLUMN ADD 65518;
    '''
    
    cur.execute(qry)

Copyright 2023, Teradata Corporation